In [1]:
from IPython.display import display, HTML

def pretty_print(df):
    return display(HTML(df.to_html().replace("\\n","<br>") ) )

In [2]:
## Import Libraries and Data

In [3]:
import glob
import dask.dataframe as dd
import pandas as pd
from pandarallel import pandarallel
import matplotlib.pyplot as plt
from datetime import datetime, timezone
import ast
import numpy as np
from operator import itemgetter
from stargazer.stargazer import Stargazer
import os
import datetime
from dateutil.rrule import rrule, MONTHLY, YEARLY, WEEKLY
from dateutil.relativedelta import relativedelta
from linearmodels.panel import PanelOLS
import multiprocessing
import statsmodels.formula.api as smf
import re
from itertools import product
import pytz


In [4]:
plt.style.use("seaborn")

In [5]:
pandarallel.initialize(progress_bar = False)

INFO: Pandarallel will run on 32 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [6]:
pd.set_option('display.max_columns', None)

In [7]:
def readDf(df,cols):
    cols = [col for col in cols if col in df.columns]
    return df[cols]

In [8]:
def readGlobDf(filename):
    df = pd.concat([pd.read_parquet(file, engine = 'pyarrow') for file in 
               glob.glob(filename.replace(".parquet","*.parquet"))])
    return df

In [9]:
def read_parquet(f, columns = None):
    try:
        if columns == None:
            return pd.read_parquet(f)
        else:
            return pd.read_parquet(f, columns = columns)
    except:
        return pd.DataFrame()

# Data that I have

## Data that links actor, organization and repository ids to logins (names)

In [10]:
df_actor = readGlobDf('data/merged_data/imputed_ranks/actor_login_id.parquet')
df_org = readGlobDf('data/merged_data/imputed_ranks/org_login_id.parquet')
df_repo = readGlobDf('data/merged_data/imputed_ranks/repo_login_id.parquet')

In [11]:
df_repo['org_login'] = df_repo['repo_name'].apply(lambda x: x.split("/")[0])

In [12]:
org_dict = pd.merge(df_repo, df_org).drop_duplicates(['repo_id'])[['repo_id', 'org_id']].set_index('repo_id').to_dict()['org_id']

## Data on when all actors first appeared

In [13]:
df_actor_start = readGlobDf('data/merged_data/imputed_ranks/allActors.parquet')

## Data on imputed ranks

In [14]:
all_ranked_v1 = readGlobDf('data/merged_data/imputed_ranks/all_ranked_v1.parquet')
all_ranked_v1['created_at'] = pd.to_datetime(all_ranked_v1['created_at'])

In [15]:
all_ranked_v2 = readGlobDf('data/merged_data/imputed_ranks/all_ranked_v2.parquet')
all_ranked_v2['created_at'] = pd.to_datetime(all_ranked_v2['created_at'])

## Data on whose committed to a repo, and aggregated data describing their email addresses

In [16]:
committer_push = readGlobDf('data/merged_data/committers_info_push.parquet')
committer_pr = readGlobDf('data/merged_data/committers_info_pr.parquet')

# note that the below is an underestimate of the # of corporate contributors (because some people use their personal emails)
committer_info = readGlobDf('data/merged_data/committer_detailed_info.parquet')
for col in ['actor_id', 'info', 'names' ,'emails','institutions','user types']:
    committer_info[col] = committer_info[col].apply(ast.literal_eval)

## Data on Issues and PRs

In [17]:
issueEventData = readGlobDf('data/merged_data/issue_data.parquet')
prEventData = readGlobDf('data/merged_data/prEventData.parquet')

In [18]:
linked_issues = readGlobDf('data/merged_data/linked_issues.parquet')

# Analysis

In [19]:
org_true = all_ranked_v2[all_ranked_v2['organization'] == False].index
all_ranked_v2.loc[org_true, 'corrected_permissions'] = all_ranked_v2.loc[org_true, 'corrected_permissions'].apply(
    lambda x: 'collaborator' if x != 'owner' else x)

In [20]:
import pytz

In [21]:
print('\nOrganization',
      all_ranked_v2[all_ranked_v2['organization'] == True].sort_values(
           'created_at', ascending = False).drop_duplicates(
          ['repo_id', 'actor_id', 'organization'])['corrected_permissions'].value_counts(normalize = True).round(2))
print('\nNot Organization',
      all_ranked_v2[all_ranked_v2['organization'] == False].sort_values(
          'created_at', ascending = False).drop_duplicates(
          ['repo_id', 'actor_id', 'organization'])['corrected_permissions'].value_counts(normalize = True).round(2))


Organization corrected_permissions
write     0.68
triage    0.32
owner     0.00
Name: proportion, dtype: float64

Not Organization corrected_permissions
collaborator    0.9
owner           0.1
Name: proportion, dtype: float64


## Analysis - think about TASKS

In [22]:
issueEventData['created_at']= pd.to_datetime(issueEventData['created_at'], format='mixed', errors = 'coerce')
issueEventData['created_at'] = issueEventData['created_at'].dt.tz_localize(pytz.UTC)

In [23]:
df_comments = issueEventData[(issueEventData['type'] == 'IssueCommentEvent')]
df_closed = issueEventData[issueEventData['issue_action'] == 'closed']

In [24]:
all_ranked_v2['permissions_date'] = all_ranked_v2.apply(lambda x: [x['corrected_permissions'], x['created_at']], axis = 1)

In [25]:
permissions_dict = all_ranked_v2[(all_ranked_v2['organization'] == True) & (all_ranked_v2['corrected_permissions'] != 'owner')].groupby(
    ['actor_id', 'repo_id']).agg({'permissions_date':lambda x: sorted(list(x), key = lambda y: y[1])}).to_dict()['permissions_date']

In [26]:
def addPermissions(lst, date):
    if type(lst) == float:
        return np.nan

    if len(lst) == 1:
        return lst[0][0]
    else:
        lst = sorted(lst, key = lambda x: x[1], reverse = True)
        for i in range(len(lst)):
            if lst[i][1].tz_localize(None)>=date.tz_localize(None):
                return lst[i][0]
        return lst[-1][0]

In [27]:
df_comments['repo_id'] = df_comments['repo_id'].apply(lambda x: org_dict.get(x, f"REPO_{x}"))
df_closed['repo_id'] = df_closed['repo_id'].apply(lambda x: org_dict.get(x, f"REPO_{x}"))
df_comments['permissions'] = df_comments.parallel_apply(lambda x: addPermissions(
    permissions_dict.get((x['actor_id'], x['repo_id']), np.nan), x['created_at']), axis = 1)
df_closed['permissions'] = df_closed.parallel_apply(lambda x: addPermissions(permissions_dict.get((x['actor_id'], x['repo_id']), np.nan), x['created_at']), axis = 1)

/tmp/user/20506/ipykernel_3096309/3024049741.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_comments['repo_id'] = df_comments['repo_id'].apply(lambda x: org_dict.get(x, f"REPO_{x}"))
/tmp/user/20506/ipykernel_3096309/3024049741.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_closed['repo_id'] = df_closed['repo_id'].apply(lambda x: org_dict.get(x, f"REPO_{x}"))
/tmp/user/20506/ipykernel_3096309/3024049741.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Da

In [28]:
df_commenting_permissions = df_comments[~df_comments['permissions'].isna()]
df_commenting_permissions['organization'] = True
df_commenting_permissions['corrected_permissions'] = df_commenting_permissions['permissions']

df_closed_permissions = df_closed[~df_closed['permissions'].isna()]
df_closed_permissions['organization'] = True
df_closed_permissions['corrected_permissions'] = df_closed_permissions['permissions']

## Commits

In [29]:
committer_push.dropna(inplace = True)
committer_pr.dropna(inplace = True)

In [30]:
committer_push['committer_info'] = committer_push['committer_info'].apply(lambda x: ast.literal_eval if type(x) != list else x)
committer_pr['committer_info'] = committer_pr['committer_info'].apply(lambda x: ast.literal_eval if type(x) != list else x)

In [31]:
committer_push['email'] = committer_push['email'].apply(lambda x: x.lower())
committer_pr['email'] = committer_pr['email'].apply(lambda x: x.lower())

In [32]:
email_committer_info = pd.concat([committer_push[['email', 'actor_id']],
                                 committer_pr[['email', 'actor_id']]]).drop_duplicates().set_index('email').to_dict()['actor_id']

In [33]:
def readCommits(file, usecols):
    return pd.read_parquet(file, usecols = usecols)

In [34]:
cols = ['repo_id', 'repo_name', 'actor_id', 'org_id', 'commit changes total','commit author name',
        'commit author email','committer name','commmitter email','commit files changed count','commit time',
       'commit additions','commit deletions','commit file changes']
cols_pr = cols.copy()
cols.extend(['push_id',])
cols_pr.extend(['pr_number'])

In [35]:
%%time
df_commits_pr = pd.concat([pd.read_parquet(file, engine = 'pyarrow') for file in 
                           glob.glob('data/github_commits/parquet/filtered_github_data/*_pr_*')])
df_commits_pr = df_commits_pr[cols_pr]
df_commits_pr['type'] = 'pr commits'

CPU times: user 1min 39s, sys: 42.3 s, total: 2min 21s
Wall time: 2min 53s


In [36]:
%%time
df_commits_push = pd.concat([pd.read_parquet(file, engine = 'pyarrow') for file in 
                           glob.glob('data/github_commits/parquet/filtered_github_data/*_push_*')])[cols]
df_commits_push['type'] = 'push commits'

CPU times: user 1min 48s, sys: 45.9 s, total: 2min 34s
Wall time: 2min 56s


In [37]:
%%time
df_commits_pr_pre = pd.concat([pd.read_parquet(file, engine = 'pyarrow') for file in 
                           glob.glob('data/github_commits/parquet/github_data_pre_18/*_pr_*')])
df_commits_pr_pre = df_commits_pr_pre[cols_pr]
df_commits_pr_pre['type']  = 'pr commits'


CPU times: user 1min 12s, sys: 22.5 s, total: 1min 34s
Wall time: 1min 41s


In [38]:
%%time
df_commits_push_pre = pd.concat([pd.read_parquet(file, engine = 'pyarrow') for file in 
                           glob.glob('data/github_commits/parquet/github_data_pre_18/*_push_*')])[cols]
df_commits_push_pre['type']  = 'push commits'

CPU times: user 1min 30s, sys: 28.7 s, total: 1min 59s
Wall time: 2min 9s


In [39]:
%%time
df_commits_pr_post = pd.concat([read_parquet(file) for file in 
                           glob.glob('data/github_commits/parquet/github_data_2324/*_pr_*')])
df_commits_pr_post = df_commits_pr_post[cols_pr]
df_commits_pr_post['type']  = 'pr commits'


CPU times: user 1min 27s, sys: 43.2 s, total: 2min 10s
Wall time: 2min 50s


In [40]:
%%time
df_commits_push_post = pd.concat([read_parquet(file) for file in 
                           glob.glob('data/github_commits/parquet/github_data_2324/*_push_*')])[cols]
df_commits_push_post['type']  = 'push commits'

CPU times: user 1min 44s, sys: 58.9 s, total: 2min 43s
Wall time: 3min 17s


In [41]:
df_commits = pd.concat([df_commits_pr, df_commits_push, df_commits_pr_pre, df_commits_push_pre,
                        df_commits_pr_post, df_commits_push_post])

In [42]:
df_commits['repo_org'] = df_commits['repo_name'].apply(lambda x: x.split("/")[0])
df_commits['organization'] = df_commits['repo_org'].isin(df_org['org_login'])

In [43]:
df_commits.reset_index(drop = True, inplace = True)

In [44]:
pushData = glob.glob('data/github_clean/filtered_github_data/pushEvent*')
pushData.extend(glob.glob('data/github_clean/github_data_pre_18/pushEvent*'))
pushData.extend(glob.glob('data/github_clean/github_data_2324/pushEvent*'))

pushEventData = pd.concat([pd.read_parquet(ele) for ele in pushData if '.parquet' in ele]) [[
    'actor_login', 'actor_id', 'org_id', 'org_login', 'type', 'created_at', 'repo_id', 'repo_name', 'push_id', 'push_ref']]
df_commits = pd.merge(df_commits, pushEventData[['push_id', 'push_ref']].drop_duplicates(), how = 'left')

In [45]:
df_commits = df_commits[['commit author email', 'commit time',
            'repo_id', 'organization', 'commit changes total', 'type', 'pr_number','push_id','push_ref',
                        'commit additions','commit deletions','commit files changed count']]

In [46]:
df_commits['commit time'] = pd.to_datetime(df_commits['commit time'], unit = 's')

In [47]:
df_commits = df_commits[~df_commits['commit author email'].isna()]
df_commits = df_commits[~df_commits['repo_id'].isna()]

In [48]:
df_commits['commit_actor_id'] = df_commits['commit author email'].apply(lambda x: email_committer_info.get(x.lower(), np.nan))

In [49]:
df_commits = df_commits[~df_commits['commit_actor_id'].isna()]

In [50]:
df_commits['commit day'] = df_commits['commit time'].apply(lambda x: datetime.datetime(x.year, x.month, x.day))

In [51]:
earliest_commit = df_commits.groupby(['commit_actor_id', 'repo_id', 'organization'])['commit time'].min()
earliest_commit = earliest_commit.reset_index()

In [52]:
earliest_commit['commit time'] = earliest_commit['commit time'].dt.tz_localize(pytz.UTC)

In [53]:
daily_commit_info = df_commits.groupby(['commit_actor_id', 'repo_id', 'commit day', 'organization']).agg({'commit changes total':['sum', 'count']})


# Thinking More Critically About Tasks

In [54]:
df_actor_start['permissions'] = 'read'

In [55]:
df_actor_all = pd.concat([df_actor_start, all_ranked_v1]).reset_index(drop = True)

In [56]:
df_actor_all = pd.merge(df_actor_all,
         all_ranked_v1.drop(['type'], axis = 1).rename({'created_at':'promoted_date', 'permissions':'corrected_permissions'}, axis = 1),
         how = 'left', on = ['repo_id', 'actor_id', 'organization'])

In [57]:
df_actor_all = df_actor_all[~df_actor_all['created_at'].isna()]

In [58]:
df_actor_all['created_at']= pd.to_datetime(df_actor_all['created_at'], format='mixed', errors = 'coerce')
#df_actor_all['created_at'] = df_actor_all['created_at'].apply(lambda x: x.astimezone(pytz.UTC))

In [59]:
# assume that if someone jumps from having read to more than read privileges in 7 days, it's bc they always had those privileges
read_to_privileged = df_actor_all[df_actor_all.apply(
    lambda x: (x['permissions'] == 'read' and 
              (x['promoted_date']-x['created_at']).days <= 7 and not pd.isnull(x['promoted_date'])), axis = 1)].index
df_actor_all.loc[read_to_privileged, 'permissions'] = df_actor_all.loc[read_to_privileged, 'corrected_permissions']

In [60]:
# assume that if someone jumps from having read to more than read privileges in 7 days, it's bc they always had those privileges
read_to_privileged = df_actor_all[df_actor_all.apply(
    lambda x: (x['permissions'] == 'read' and 
              (x['promoted_date']-x['created_at']).days <= 7 and not pd.isnull(x['promoted_date'])), axis = 1)].index
df_actor_all.loc[read_to_privileged, 'permissions'] = df_actor_all.loc[read_to_privileged, 'corrected_permissions']

In [61]:
df_actor_all = df_actor_all.sort_values('created_at').drop_duplicates(['repo_id', 'actor_id', 'organization', 'permissions']).drop(
    ['type','promoted_date','corrected_permissions'], axis = 1)

In [62]:
perm_dict = {'read': 4, 'triage': 3, 'write': 2, 'owner': 1}
df_actor_all['permissions_ranked'] = df_actor_all['permissions'].apply(lambda x: perm_dict[x])

In [63]:
df_actor_all = df_actor_all.sort_values(['created_at', 'permissions']).drop_duplicates(
    ['created_at', 'repo_id', 'actor_id', 'organization'])

# Analysis

In [64]:
python_opensource = pd.read_table('data/inputs/README-Python.md', sep="|", header=0, index_col=0, skipinitialspace=True, on_bad_lines='warn').dropna(axis=1, how='all').iloc[1:]   
python_opensource['repo_name'] = python_opensource['NAME/PLACE'].apply(lambda x: "/".join(x.split("(")[1].replace(")","").split("/")[-3:-1]))

Skipping line 145: expected 8 fields, saw 10
Skipping line 441: expected 8 fields, saw 11
Skipping line 516: expected 8 fields, saw 9
Skipping line 630: expected 8 fields, saw 10
Skipping line 974: expected 8 fields, saw 9



In [65]:
python_opensource = pd.merge(df_repo, python_opensource)
repo_ids = python_opensource['repo_id'].tolist()
df_actor_all['Free_Copilot'] = df_actor_all.apply(lambda x: x['repo_id'] in repo_ids and x['permissions'] in ['owner', 'write'] and 
                                                  x['created_at']<datetime.datetime(2022, 6, 23, tzinfo = pytz.UTC), axis = 1)
free_cp = df_actor_all[df_actor_all['Free_Copilot'] == True]['actor_id'].tolist()
df_actor_all['Free_Copilot']  = df_actor_all.apply(lambda x: x['actor_id'] in free_cp, axis = 1)

In [66]:
bots = pd.to_numeric(df_actor[df_actor['actor_login'].apply(lambda x: '[bot]' in x)]['actor_id']).unique()
df_actor_all = df_actor_all[~df_actor_all['actor_id'].isin(bots)]

## Population Data

In [67]:
# test population as including triagers LOL
special_actors = df_actor_all[df_actor_all['permissions'].isin(['owner','write'])]['actor_id'].unique().tolist()
population = df_actor_all[df_actor_all['actor_id'].isin(special_actors)]
# population = df_actor_all[df_actor_all['permissions'].isin(['owner','write', 'triage'])]
# population = df_actor_all# [df_actor_all['permissions'].isin(['owner','write', 'triage'])]

population['Free_Copilot'] = population['Free_Copilot'].astype(int)
population = population[population['created_at']<=datetime.datetime(2021,6,23,tzinfo=pytz.UTC)]
population['key'] = population['repo_id'].apply(lambda x: str(int(x))) + "_" + population['actor_id'].apply(lambda x: str(int(x)))

/tmp/user/20506/ipykernel_3096309/1449098524.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  population['Free_Copilot'] = population['Free_Copilot'].astype(int)


In [68]:
watch_parquet = glob.glob('data/github_clean/filtered_github_data/watchEvent*.parquet')
watch_parquet.extend(glob.glob('data/github_clean/github_data_pre_18/watchEvent*.parquet'))
watch_parquet.extend(glob.glob('data/github_clean/github_data_2324/watchEvent*.parquet'))
df_watch_parquet = pd.concat([read_parquet(ele, ['created_at', 'repo_id']) for ele in watch_parquet])
df_watch_parquet['created_at'] = pd.to_datetime(df_watch_parquet['created_at'], format='mixed')
df_watch = df_watch_parquet

fork_parquet = glob.glob('data/github_clean/filtered_github_data/forkEvent*.parquet')
fork_parquet.extend(glob.glob('data/github_clean/github_data_pre_18/forkEvent*.parquet'))
fork_parquet.extend(glob.glob('data/github_clean/github_data_2324/forkEvent*.parquet'))
df_fork_parquet = pd.concat([read_parquet(ele, ['created_at', 'repo_id']) for ele in fork_parquet])
df_fork_parquet['created_at'] = pd.to_datetime(df_fork_parquet['created_at'])
df_fork = df_fork_parquet

In [69]:
repos2000 = df_watch['repo_id'].value_counts()[df_watch['repo_id'].value_counts()>2000].index
repos1000 = df_watch['repo_id'].value_counts()[df_watch['repo_id'].value_counts()>1000].index
repos500 = df_watch['repo_id'].value_counts()[df_watch['repo_id'].value_counts()>500].index

In [70]:
population['greater_2000_stars'] = population['repo_id'].isin(repos2000)
population['greater_1000_stars'] = population['repo_id'].isin(repos1000)
population['greater_500_stars'] = population['repo_id'].isin(repos500)

## Controls

In [71]:
df_fork['created_at'] = pd.to_datetime(df_fork['created_at'])
df_watch['created_at'] = pd.to_datetime(df_watch['created_at'])

ValueError: Cannot mix tz-aware with tz-naive values, at position 926995

In [ ]:
df_fork['created_at_month_year'] = df_fork['created_at'].apply(
    lambda x: datetime.datetime(x.year, x.month, 28, tzinfo = pytz.UTC))
df_watch['created_at_month_year'] = df_watch['created_at'].apply(
    lambda x: datetime.datetime(x.year, x.month, 28, tzinfo = pytz.UTC))
df_watch_fork = pd.concat([df_watch.groupby(['created_at_month_year', 'repo_id']).count(),
                           df_fork.groupby(['created_at_month_year', 'repo_id']).count()], axis = 1)
df_watch_fork.columns = ['stars', 'forks']
df_watch_fork = pd.concat([df_watch_fork.fillna(0).reset_index().sort_values(
    ['repo_id', 'created_at_month_year']),
               df_watch_fork.fillna(0).reset_index().sort_values(
                   ['repo_id', 'created_at_month_year']).groupby(
                   ['repo_id'])[['stars','forks']].transform('cumsum')], axis = 1)
df_watch_fork.columns = ['created_at_month_year', 'repo_id', 'gained stars', 'gained forks', 'cumulative stars', 'cumulative forks']

In [ ]:
df_repo_issue = issueEventData[issueEventData['issue_action'].isin(['opened', None])].sort_values(
    'created_at').drop_duplicates(['repo_id','issue_number'])
df_repo_issue['created_at_month_year'] = df_repo_issue['created_at'].apply(
    lambda x: datetime.datetime(x.year, x.month, 28, tzinfo = pytz.UTC))
df_repo_pr = prEventData[prEventData['pr_action'].isin(['opened', None])].sort_values(
    'created_at').drop_duplicates(['repo_id','pr_number'])
df_repo_pr = df_repo_pr[~df_repo_pr['created_at'].isna()]
df_repo_pr['created_at_month_year'] = df_repo_pr['created_at'].apply(
    lambda x: datetime.datetime(x.year, x.month, 28, tzinfo = pytz.UTC))
df_issue_pr = pd.concat([df_repo_issue.groupby(['created_at_month_year', 'repo_id'])['type'].count(),
                           df_repo_pr.groupby(['created_at_month_year', 'repo_id'])['type'].count()], axis = 1)
df_issue_pr.columns = ['opened issues', 'opened prs']
df_issue_pr = pd.concat([df_issue_pr.fillna(0).reset_index().sort_values(
    ['repo_id', 'created_at_month_year']),
               df_issue_pr.fillna(0).reset_index().sort_values(
                   ['repo_id', 'created_at_month_year']).groupby(
                   ['repo_id'])[['opened issues','opened prs']].transform('cumsum')], axis = 1)
df_issue_pr.columns = ['created_at_month_year', 'repo_id', 'opened issues','opened prs', 'cumulative opened issues','cumulative opened prs']
                         

In [ ]:
all_ranked_v1['created_at_month_year'] = all_ranked_v1['created_at'].apply(
    lambda x: datetime.datetime(x.year, x.month, 28, tzinfo = pytz.UTC))

df_ranks = pd.concat([all_ranked_v1[all_ranked_v1['permissions']=='triage'].groupby(
    ['created_at_month_year','repo_id'])['organization'].count(),
                         all_ranked_v1[all_ranked_v1['permissions'].isin(['triage','write','owner'])].groupby(
    ['created_at_month_year','repo_id'])['organization'].count(),], axis = 1)
df_ranks.columns = ['triagers', 'writers']
df_ranks_cum = pd.concat([df_ranks.fillna(0).reset_index().sort_values(
    ['repo_id', 'created_at_month_year']),
               df_ranks.fillna(0).reset_index().sort_values(
                   ['repo_id', 'created_at_month_year']).groupby(
                   ['repo_id'])[['triagers','writers']].transform('cumsum')], axis = 1)
df_ranks_cum.columns = ['created_at_month_year', 'repo_id', 'added triagers','added writers', 'cumulative triagers','cumulative writers']


In [ ]:
df_repo_controls = pd.merge(pd.merge(df_watch_fork, df_issue_pr, how = 'left'), df_ranks_cum, how = 'left').fillna(0)

In [ ]:
df_commits['created_at_month_year'] = df_commits['commit day'].apply(
    lambda x: datetime.datetime(x.year, x.month, 28, tzinfo = pytz.UTC))

In [ ]:
df_contributing_count = df_commits[['repo_id', 'commit_actor_id', 'created_at_month_year']].drop_duplicates().groupby(
    ['commit_actor_id','created_at_month_year']).count()
df_contributing_count = df_contributing_count.reset_index()
df_contributing_count.rename({'repo_id': 'commit_projects'}, axis = 1, inplace = True)
df_commits_user = df_commits[['commit_actor_id', 'created_at_month_year']].value_counts()


In [ ]:
df_repo_issue_user = issueEventData[issueEventData['issue_action'].isin(['opened', None])].sort_values(
    'created_at').drop_duplicates(['actor_id','issue_number'])
df_repo_issue_user['created_at_month_year'] = df_repo_issue['created_at'].apply(
    lambda x: datetime.datetime(x.year, x.month, 28, tzinfo = pytz.UTC))
df_repo_pr_user = prEventData[prEventData['pr_action'].isin(['opened', None])].sort_values(
    'created_at').drop_duplicates(['actor_id','pr_number'])
df_repo_pr_user = df_repo_pr[~df_repo_pr['created_at'].isna()]
df_repo_pr_user['created_at_month_year'] = df_repo_pr['created_at'].apply(
    lambda x: datetime.datetime(x.year, x.month, 28, tzinfo = pytz.UTC))
df_issue_pr_user = pd.concat([df_repo_issue_user.groupby(['created_at_month_year', 'actor_id'])['type'].count(),
                           df_repo_pr_user.groupby(['created_at_month_year', 'actor_id'])['type'].count()], axis = 1)
df_issue_pr_user.columns = ['opened issues', 'opened prs']
df_issue_pr_user = pd.concat([df_issue_pr_user.fillna(0).reset_index().sort_values(
    ['actor_id', 'created_at_month_year']),
               df_issue_pr_user.fillna(0).reset_index().sort_values(
                   ['actor_id', 'created_at_month_year']).groupby(
                   ['actor_id'])[['opened issues','opened prs']].transform('cumsum')], axis = 1)
df_issue_pr_user.columns = ['created_at_month_year', 'actor_id', 'user opened issues','user opened prs', 'cumulative user opened issues','cumulative user opened prs']
                         

In [ ]:
df_ranks_user = pd.concat([all_ranked_v1[all_ranked_v1['permissions']=='triage'].groupby(
    ['created_at_month_year','actor_id'])['organization'].count(),
                         all_ranked_v1[all_ranked_v1['permissions'].isin(['triage','write','owner'])].groupby(
    ['created_at_month_year','actor_id'])['organization'].count(),], axis = 1)
df_ranks_user.columns = ['triagers', 'writers']
df_ranks_user_cum = pd.concat([df_ranks_user.fillna(0).reset_index().sort_values(
    ['actor_id', 'created_at_month_year']),
               df_ranks_user.fillna(0).reset_index().sort_values(
                   ['actor_id', 'created_at_month_year']).groupby(
                   ['actor_id'])[['triagers','writers']].transform('cumsum')], axis = 1)
df_ranks_user_cum.columns = ['created_at_month_year', 'actor_id', 'user added triagers','user added writers', 'user cumulative triagers','user cumulative writers']


In [ ]:
contributed_projects_forks_watch = pd.merge(df_commits[['repo_id', 'commit_actor_id', 'created_at_month_year']].drop_duplicates(),
                                            df_watch_fork).groupby(['commit_actor_id','created_at_month_year'])[
    ['gained stars','gained forks','cumulative stars','cumulative forks']].sum().reset_index()
contributed_projects_forks_watch.columns = [
    'commit_actor_id', 'created_at_month_year','user gained stars','user gained forks','user cumulative stars','user cumulative forks']

In [ ]:
df_contributor_controls = pd.concat([df_commits_user.reset_index().set_index(['commit_actor_id','created_at_month_year']).rename(
    {'count':'commit count'}, axis = 1),
           df_contributing_count.set_index(['commit_actor_id','created_at_month_year']),
           df_issue_pr_user.set_index(['actor_id','created_at_month_year']),
           df_ranks_user_cum.set_index(['actor_id','created_at_month_year']),
           #contributed_projects_forks_watch.set_index(['commit_actor_id','created_at_month_year'])
                                    ], axis = 1)
df_contributor_controls = df_contributor_controls.reset_index().rename({'level_0':'commit_actor_id'}, axis = 1)

In [ ]:
df_repo_controls
df_contributor_controls

## DiD - Commits

In [ ]:
pr_status_info = prEventData[['created_at', 'pr_number', 'repo_id', 'pr_action']].sort_values(
    'created_at', ascending = True).dropna().drop_duplicates(['pr_number', 'repo_id', 'pr_action'])

In [ ]:
df_commits['key'] = df_commits['repo_id'].apply(lambda x: str(int(x))) + "_" + df_commits['commit_actor_id'].apply(lambda x: str(int(x)))


df_copilot_commits = df_commits[df_commits['key'].isin(population['key'].tolist())]
df_copilot_commits['created_at_month_year'] = df_copilot_commits['commit time'].apply(
    lambda x: datetime.datetime(x.year, x.month, 28, tzinfo = pytz.UTC))
df_copilot_commits = df_copilot_commits[
    (df_copilot_commits['created_at_month_year']>= datetime.datetime(2021, 6, 23, tzinfo = pytz.UTC))]

pr_status_dict = pr_status_info[['pr_number','repo_id','pr_action']].set_index(['pr_number','repo_id']).to_dict()['pr_action']
df_copilot_commits['pr_status'] = df_copilot_commits.apply(lambda x: pr_status_dict.get((x['pr_number'], x['repo_id']), np.nan), axis = 1)

df_copilot_commits = pd.merge(df_copilot_commits, prEventData[['repo_id', 'pr_number','pr_merged_by_id']].dropna(), how = 'left')
df_copilot_commits['pr_merged'] = (~df_copilot_commits['pr_merged_by_id'].isna()).astype(int)
df_copilot_commits['push_ref'] = (df_copilot_commits['push_ref'].isin(['refs/heads/master', 'refs/heads/main'])).astype(int)
df_copilot_commits['commit changes total pr merged'] = df_copilot_commits.apply(
    lambda x: x['commit changes total']*x['pr_merged'], axis = 1)
df_copilot_commits['commit changes total push main'] = df_copilot_commits.apply(
    lambda x: x['commit changes total']*x['push_ref'], axis = 1)
df_copilot_commits['commit changes added pr merged'] = df_copilot_commits.apply(
    lambda x: x['commit additions']*x['pr_merged'], axis = 1)
df_copilot_commits['commit changes added push main'] = df_copilot_commits.apply(
    lambda x: x['commit additions']*x['push_ref'], axis = 1)
df_copilot_commits['commit changes deleted pr merged'] = df_copilot_commits.apply(
    lambda x: x['commit deletions']*x['pr_merged'], axis = 1)
df_copilot_commits['commit changes deleted push main'] = df_copilot_commits.apply(
    lambda x: x['commit deletions']*x['push_ref'], axis = 1)


In [ ]:
df_copilot_commits_grouped =  df_copilot_commits.groupby(['commit_actor_id', 'created_at_month_year', 'repo_id', 'type']).agg(
    {'commit changes total':['sum', 'count'],
     'commit additions': 'sum',
     'commit deletions': 'sum',
     'commit files changed count': 'sum',
     'pr_merged':'sum','push_ref':'sum',
     'commit changes total pr merged': 'sum',
     'commit changes total push main': 'sum',
     'commit changes added pr merged': 'sum',
     'commit changes added push main': 'sum',
     'commit changes deleted pr merged': 'sum',
     'commit changes deleted push main': 'sum'}, ).reset_index()



In [ ]:
df_copilot_commits_grouped.columns =  ['commit_actor_id', 'created_at_month_year', 'repo_id','type',
     'commit_changes_sum', 'commit_count', 'commit_additions_sum', 'commit_deletions_sum',
     'commit_files_changed_sum', 'pr_merged_sum', 'push_main_sum','commit_changes_pr_merged_sum',
     'commit_changes_push_main_sum', 'commit_additions_pr_merged_sum', 'commit_additions_push_main_sum',
     'commit_deletions_pr_merged_sum', 'commit_deletions_push_main_sum',]
df_copilot_commits_grouped = df_copilot_commits_grouped.pivot(
    index = ['commit_actor_id', 'created_at_month_year','repo_id'],
    columns =  'type', values = ['commit_changes_sum', 'commit_count', 'commit_additions_sum', 'commit_deletions_sum',
                                 'commit_files_changed_sum','pr_merged_sum', 'push_main_sum','commit_changes_pr_merged_sum',
                                'commit_changes_push_main_sum', 'commit_additions_pr_merged_sum', 'commit_additions_push_main_sum',
                                 'commit_deletions_pr_merged_sum', 'commit_deletions_push_main_sum',]).reset_index()




In [ ]:
df_copilot_commits_grouped = df_copilot_commits_grouped.drop(
    [('pr_merged_sum','push commits'), ('push_main_sum', 'pr commits'), ('commit_changes_push_main_sum','pr commits'),
     ('commit_changes_pr_merged_sum','push commits'), ('commit_additions_push_main_sum','pr commits'),
     ('commit_additions_pr_merged_sum','push commits'), ('commit_deletions_push_main_sum','pr commits'),
     ('commit_deletions_pr_merged_sum','push commits')], axis = 1)

In [ ]:
df_copilot_commits_grouped.columns =  ['commit_actor_id', 'created_at_month_year', 'repo_id' ,'commit_changes_sum_pr','commit_changes_sum_push', 
     'commit_count_pr','commit_count_push', 'commit_additions_sum_pr','commit_additions_sum_push', 
     'commit_deletions_sum_pr','commit_deletions_sum_push', 'commit_files_changed_sum_pr',
     'commit_files_changed_sum_push', 'pr_merged_sum', 'push_main_sum','commit_changes_pr_merged_sum',
     'commit_changes_push_main_sum', 'commit_additions_pr_merged_sum', 'commit_additions_push_main_sum',
     'commit_deletions_pr_merged_sum', 'commit_deletions_push_main_sum']





In [ ]:
df_copilot_commits_grouped = pd.merge(
    df_copilot_commits_grouped,
    population[['actor_id', 'repo_id', 'Free_Copilot','greater_1000_stars','greater_500_stars']].rename({'actor_id':'commit_actor_id', 'Free_Copilot':'treatment'}, axis = 1).drop_duplicates())
#df_copilot_commits_grouped = pd.merge(df_copilot_commits_grouped, df_repo_controls)
#df_copilot_commits_grouped = pd.merge(df_copilot_commits_grouped, df_contributor_controls)


In [ ]:
df_copilot_commits_grouped.fillna(0).to_csv('results/data/df_copilot_commits_grouped.csv')


In [ ]:
issueEventData['issue_pull_request_number'] = issueEventData['issue_pull_request'].apply(lambda x: ast.literal_eval(x)['html_url'] if not pd.isnull(x) else x)

In [ ]:
issueEventData['issue_pull_request_number'] = pd.to_numeric(issueEventData['issue_pull_request_number'].apply(lambda x: x.split("/")[-1] if not pd.isnull(x) else x))

In [ ]:
prEventData_na = prEventData[(~prEventData['repo_id'].isna()) & 
    (prEventData.apply(lambda x: not pd.isnull(x['pr_user_id']) or x['pr_action'] == 'opened', axis= 1))]
prEventData_na['key'] = prEventData_na.apply(
    lambda x: str(int(x['repo_id']))+"_"+
    str(int(x['pr_user_id'])) if x['pr_action'] != 'opened' else str(int(x['actor_id'])), axis = 1)

prEventData_copilot = prEventData_na[prEventData_na['key'].isin(population['key'].tolist())]
prEventData_copilot['created_at_month_year'] = prEventData_copilot['created_at'].apply(
    lambda x: datetime.datetime(x.year, x.month, 28, tzinfo = pytz.UTC))

linked_prs = issueEventData[['repo_id','issue_pull_request_number']].dropna().drop_duplicates()
linked_prs['linked_pr'] = 1
prEventData_copilot = pd.merge(prEventData_copilot, linked_prs.rename({'issue_pull_request_number':'pr_number'}, axis = 1), how = 'left')
prEventData_copilot['linked_pr'] = prEventData_copilot['linked_pr'].fillna(0)

prEventData_copilot = pd.merge(prEventData_copilot.drop('pr_merged_by_id', axis = 1), 
                                   prEventData[['repo_id', 'pr_number','pr_merged_by_id']].drop_duplicates(), how = 'left')
prEventData_copilot['merged_pr'] = prEventData_copilot['pr_merged_by_id'].apply(lambda x: not pd.isnull(x)).astype(int)


In [ ]:
pr_commit_data = df_commits[['repo_id', 'created_at_month_year', 'commit changes total', 'commit additions','commit deletions']].groupby(
    ['repo_id', 'created_at_month_year']).sum().reset_index()

In [ ]:
prEventData_copilot = pd.merge(prEventData_copilot,pr_commit_data,how = 'left')

In [ ]:
prEventData_copilot['commit_changes_merged_prs'] = prEventData_copilot.apply(
    lambda x: x['commit changes total'] * (1-int(pd.isnull(x['pr_merged_by_id']))), axis = 1)
prEventData_copilot['commit_additions_merged_prs'] = prEventData_copilot.apply(
    lambda x: x['commit additions'] * (1-int(pd.isnull(x['pr_merged_by_id']))), axis = 1)
prEventData_copilot['commit_deletions_merged_prs'] = prEventData_copilot.apply(
    lambda x: x['commit deletions'] * (1-int(pd.isnull(x['pr_merged_by_id']))), axis = 1)

In [ ]:
df_copilot_prs_grouped =  prEventData_copilot[['pr_user_id', 'repo_id', 'pr_number', 'created_at', 
                                               'created_at_month_year','linked_pr','merged_pr',
                                              'commit changes total', 'commit additions','commit deletions',
                                              'commit_changes_merged_prs','commit_additions_merged_prs','commit_deletions_merged_prs']].sort_values(
    'created_at').drop_duplicates(
    ['pr_user_id', 'repo_id', 'pr_number',]).groupby(
    ['pr_user_id', 'created_at_month_year', 'repo_id']).agg(
    {'pr_number':['count'], 'linked_pr':['sum'], 'merged_pr':'sum', 
    'commit changes total':'sum', 'commit additions':'sum','commit deletions':'sum',
     'commit_changes_merged_prs':'sum', 'commit_additions_merged_prs':'sum','commit_deletions_merged_prs':'sum'}).reset_index()
df_copilot_prs_grouped.columns = ['pr_user_id','created_at_month_year','repo_id','pr_number_count','linked_pr_sum','merged_pr_sum',
                                 'pr_commit_changes','pr_commit_additions','pr_commit_deletions',
                                 'merged_pr_commit_changes','merged_pr_commit_additions','merged_pr_commit_deletions']

In [ ]:
df_copilot_prs_grouped = pd.merge(df_copilot_prs_grouped, 
         population[['actor_id', 'repo_id',  'Free_Copilot','greater_1000_stars','greater_500_stars']].rename({'actor_id':'pr_user_id', 'Free_Copilot':'treatment'}, axis = 1).drop_duplicates())
#df_copilot_prs_grouped = pd.merge(df_copilot_prs_grouped, df_repo_controls)
#df_copilot_prs_grouped = pd.merge(df_copilot_prs_grouped, df_contributor_controls.rename({'commit_actor_id':'pr_user_id'}, axis = 1))

In [ ]:
df_copilot_prs_grouped.fillna(0).to_csv('results/data/df_copilot_prs_grouped.csv')

In [ ]:
df_commits['push_ref_sum'] = df_commits['push_ref'].apply(lambda x: x in ['refs/heads/master', 'refs/heads/main'])
df_commits['push_ref_changes'] = pd.to_numeric(df_commits['push_ref_sum'] * df_commits['commit changes total'])
df_commits['push_ref_additions'] = pd.to_numeric(df_commits['push_ref_sum'] * df_commits['commit additions'])
df_commits['push_ref_deletions'] = pd.to_numeric(df_commits['push_ref_sum'] * df_commits['commit deletions'])

df_push = df_commits[
    ['push_id', 'commit_actor_id', 'commit time', 'repo_id', 'push_ref_sum','push_ref_changes','push_ref_additions','push_ref_deletions',
    'commit changes total','commit additions', 'commit deletions']].dropna().sort_values('commit time').drop_duplicates(
    ['push_id', 'commit_actor_id', 'repo_id']).groupby(
    ['commit_actor_id','repo_id', 'commit time']).agg(
    {'push_id':'count', 'push_ref_sum':'sum','push_ref_changes':'sum', 'push_ref_additions':'sum', 'push_ref_deletions':'sum',
     'commit changes total':'sum', 'commit additions':'sum','commit deletions':'sum', }).reset_index()

In [ ]:
df_push['key'] = df_push.apply(
    lambda x: str(int(x['repo_id']))+"_"+str(int(x['commit_actor_id'])), axis = 1)

df_push = df_push[df_push['key'].isin(population['key'].tolist())]
df_push['created_at_month_year'] = df_push['commit time'].apply(
    lambda x: datetime.datetime(x.year, x.month, 28, tzinfo = pytz.UTC))
df_push.rename({'push_id':'push_count', 'push_ref_sum':'push_main_count'}, axis = 1, inplace = True)

In [ ]:
df_copilot_push_grouped = pd.merge(
    df_push.groupby(['created_at_month_year', 'commit_actor_id', 'repo_id'])[
    ['push_count','push_main_count','commit changes total','commit additions','commit deletions',
     'push_ref_changes','push_ref_additions','push_ref_deletions',]].sum().reset_index(),
    population[['actor_id',  'repo_id', 'Free_Copilot','greater_1000_stars','greater_500_stars']].rename(
        {'actor_id':'commit_actor_id', 'Free_Copilot':'treatment'}, axis = 1).drop_duplicates())
#df_copilot_push_grouped = pd.merge(df_copilot_push_grouped, df_repo_controls)
#df_copilot_push_grouped = pd.merge(df_copilot_push_grouped, df_contributor_controls)

In [ ]:
df_copilot_push_grouped.fillna(0).to_csv('results/data/df_copilot_push_grouped.csv')

In [ ]:
prEventDataMerged = prEventData[~prEventData['pr_merged_by_login'].isna()][['repo_id', 'pr_number']].drop_duplicates()

In [ ]:
pr_commits_earliest = df_commits.groupby(['repo_id', 'pr_number']).agg({'commit time':['min', 'count']})

In [ ]:
df_pr_lengths = pd.merge(prEventData,prEventDataMerged).groupby(['repo_id', 'actor_id','pr_number', 'pr_action'])['created_at'].min().reset_index()

In [ ]:
df_pr_lengths['created_at'] = df_pr_lengths['created_at'].astype(str)
df_pr_lengths_wide = df_pr_lengths.pivot(index = ['repo_id', 'actor_id', 'pr_number'], columns = 'pr_action',values = 'created_at')
df_pr_lengths_wide = df_pr_lengths_wide[df_pr_lengths_wide.columns[:2]].dropna().reset_index()
df_pr_lengths_wide.columns = ['repo_id', 'actor_id', 'pr_number', 'closed', 'opened']
df_pr_lengths_wide['closed'] = pd.to_datetime(df_pr_lengths_wide['closed'])
df_pr_lengths_wide['opened']  = pd.to_datetime(df_pr_lengths_wide['opened'])
df_commits_pr_grouped = df_commits.groupby(['pr_number', 'repo_id']).agg(
    {'commit time':['count', 'min', 'max'], 'commit changes total':'sum', }).reset_index().drop_duplicates()
df_commits_pr_grouped.columns = ['pr_number', 'repo_id', 'commits', 'earliest_date', 'latest_date', 'commit changes']

df_pr_lengths_wide = pd.merge(df_pr_lengths_wide, df_commits_pr_grouped)

In [ ]:
pr_commits_earliest = pr_commits_earliest.reset_index()
pr_commits_earliest.columns = ['repo_id', 'pr_number', 'earliest_commit_time', 'commit_count']

In [ ]:
df_pr_lengths_wide = pd.merge(df_pr_lengths_wide,pr_commits_earliest)
df_pr_lengths_wide['hours'] = (df_pr_lengths_wide['closed']-df_pr_lengths_wide['opened']).apply(lambda x: x.total_seconds()/3600)
df_pr_lengths_wide['opened'] = pd.to_datetime(df_pr_lengths_wide['opened'], utc = True)
df_pr_lengths_wide['earliest_commit_time'] = pd.to_datetime(df_pr_lengths_wide['earliest_commit_time'], utc = True)
df_pr_lengths_wide['created_at_month_year'] = np.minimum(df_pr_lengths_wide['opened'],df_pr_lengths_wide['earliest_commit_time']).apply(
    lambda x: datetime.datetime(x.year, x.month, 28, tzinfo = pytz.UTC))

In [ ]:
df_pr_lengths_wide['hours_per_commit'] = df_pr_lengths_wide['hours']/df_pr_lengths_wide['commits']
df_pr_lengths_wide['hours_per_commit_change'] = df_pr_lengths_wide['hours']/df_pr_lengths_wide['commit changes']

In [ ]:
df_pr_lengths_wide['code_hours'] = (df_pr_lengths_wide['latest_date']-df_pr_lengths_wide['earliest_date']).apply(lambda x: x.total_seconds()/3600)
df_pr_lengths_wide['code_hours_per_commit'] = df_pr_lengths_wide['code_hours']/df_pr_lengths_wide['commits']
df_pr_lengths_wide['code_hours_per_commit_change'] = df_pr_lengths_wide['code_hours'] /df_pr_lengths_wide['commit changes']

In [ ]:
df_pr_lengths_agg = df_pr_lengths_wide.groupby(['repo_id','actor_id','created_at_month_year']).agg(
    {'hours':'mean','hours_per_commit':'mean','hours_per_commit_change':'mean',
     'commit_count':'sum', 'commit changes':'sum',
    'code_hours':'mean','code_hours_per_commit':'mean', 'code_hours_per_commit_change':'mean',}).reset_index()

In [ ]:
df_copilot_pr_lengths_grouped = pd.merge(
    df_pr_lengths_agg, 
    population[['actor_id',  'repo_id', 'Free_Copilot','greater_1000_stars','greater_500_stars']].rename({'Free_Copilot':'treatment'}, axis = 1).drop_duplicates())
#df_copilot_pr_lengths_grouped = pd.merge(df_copilot_pr_lengths_grouped, df_repo_controls)
#df_copilot_pr_lengths_grouped = pd.merge(df_copilot_pr_lengths_grouped, df_contributor_controls.rename({'commit_actor_id':'actor_id'}, axis = 1))


In [ ]:
df_copilot_pr_lengths_grouped.fillna(0).to_csv('results/data/df_copilot_pr_lengths_grouped.csv')

In [ ]:
pushEventData['created_at'] = pd.to_datetime(pushEventData['created_at'], utc = True)

In [ ]:
df_push_lengths

In [ ]:
df_push_lengths = pd.merge(df_commits[df_commits['type'] == 'push commits'][['repo_id', 'push_id','commit time', 'commit changes total', 'commit files changed count']].rename({'commit time':'earliest_date'},axis= 1),
                           pushEventData[['push_id','repo_id', 'actor_id','created_at', 'push_ref']]).groupby(
    ['repo_id', 'actor_id','push_id', 'push_ref']).agg(
    {'created_at': 'min', 'earliest_date':['min','max'], 'commit changes total':'sum','commit files changed count':'count'}).reset_index()

df_push_lengths.columns = [
    'repo_id', 'actor_id','push_id', 'push_ref', 'created_at','earliest_date','latest_date','commit changes total','commit files changed count']
df_push_lengths['created_at'] = pd.to_datetime(df_push_lengths['created_at'], utc = True)
df_push_lengths['earliest_date'] = df_push_lengths['earliest_date'].dt.tz_localize(pytz.UTC)
df_push_lengths = df_push_lengths[(df_push_lengths['created_at']-df_push_lengths['earliest_date']).dt.days>0]

In [ ]:
df_push_lengths['hours'] = (df_push_lengths['created_at']-df_push_lengths['earliest_date']).dt.total_seconds()/3600
df_push_lengths['hours_per_commit'] = (df_push_lengths['hours'])/df_push_lengths['commit files changed count']
df_push_lengths['hours_per_commit_change'] = (df_push_lengths['hours'])/df_push_lengths['commit changes total']

In [ ]:
df_copilot_push_lengths = pd.merge(df_push_lengths, 
                                   population[['actor_id', 'Free_Copilot']].rename({'Free_Copilot':'treatment'}, axis = 1).drop_duplicates())
df_copilot_push_lengths['created_at_month_year'] = df_copilot_push_lengths['earliest_date'].apply(
    lambda x: datetime.datetime(x.year, x.month, 28, tzinfo = pytz.UTC))

In [ ]:
df_copilot_push_lengths_grouped = df_copilot_push_lengths.groupby(
    ['repo_id','actor_id', 'created_at_month_year'])[['hours','hours_per_commit','hours_per_commit_change',]].mean().reset_index()
df_copilot_push_lengths_grouped = pd.merge(df_copilot_push_lengths_grouped, 
         population[['actor_id', 'repo_id',  'Free_Copilot','greater_1000_stars','greater_500_stars']].rename({'Free_Copilot':'treatment'}, axis = 1).drop_duplicates())
#df_copilot_push_lengths_grouped = pd.merge(df_copilot_push_lengths_grouped, df_repo_controls)
#df_copilot_push_lengths_grouped = pd.merge(df_copilot_push_lengths_grouped, df_contributor_controls.rename({'commit_actor_id':'actor_id'}, axis = 1))
df_copilot_push_lengths_grouped.fillna(0).to_csv('results/data/df_copilot_push_lengths_grouped.csv')

In [ ]:
df_copilot_push_lengths_grouped_main = df_copilot_push_lengths[df_copilot_push_lengths['push_ref'].isin(['refs/heads/master','refs/heads/main'])].groupby(
    ['repo_id','actor_id', 'created_at_month_year'])[['hours','hours_per_commit','hours_per_commit_change',]].mean().reset_index()
df_copilot_push_lengths_grouped_main = pd.merge(df_copilot_push_lengths_grouped_main, 
         population[['actor_id', 'repo_id',  'Free_Copilot','greater_1000_stars','greater_500_stars']].rename({'Free_Copilot':'treatment'}, axis = 1).drop_duplicates())
#df_copilot_push_lengths_grouped = pd.merge(df_copilot_push_lengths_grouped, df_repo_controls)
#df_copilot_push_lengths_grouped = pd.merge(df_copilot_push_lengths_grouped, df_contributor_controls.rename({'commit_actor_id':'actor_id'}, axis = 1))
df_copilot_push_lengths_grouped_main.fillna(0).to_csv('results/data/df_copilot_push_lengths_main_grouped.csv')



In [ ]:
df_copilot_push_lengths_grouped_notmain = df_copilot_push_lengths[~df_copilot_push_lengths['push_ref'].isin(['refs/heads/master','refs/heads/main'])].groupby(
    ['repo_id','actor_id', 'created_at_month_year'])[['hours','hours_per_commit','hours_per_commit_change',]].mean().reset_index()
df_copilot_push_lengths_grouped_notmain = pd.merge(df_copilot_push_lengths_grouped_notmain, 
         population[['actor_id',  'repo_id', 'Free_Copilot','greater_1000_stars','greater_500_stars']].rename({'Free_Copilot':'treatment'}, axis = 1).drop_duplicates())
#df_copilot_push_lengths_grouped = pd.merge(df_copilot_push_lengths_grouped, df_repo_controls)
#df_copilot_push_lengths_grouped = pd.merge(df_copilot_push_lengths_grouped, df_contributor_controls.rename({'commit_actor_id':'actor_id'}, axis = 1))
df_copilot_push_lengths_grouped_notmain.fillna(0).to_csv('results/data/df_copilot_push_lengths_notmain_grouped.csv')



In [ ]:
df_all_data = pd.concat([df_copilot_commits_grouped.fillna(0).rename({'commit_actor_id':'actor_id'}, axis = 1).set_index(
    ['actor_id','created_at_month_year','repo_id']).drop([
               'treatment','greater_1000_stars','greater_500_stars'], axis = 1),
           df_copilot_push_lengths_grouped.fillna(0).rename(
               {'hours':'avg_hours_per_push', 'hours_per_commit':'avg_hours_per_push_commit', 
                'hours_per_commit_change':'avg_hours_per_push_commit_change'}, axis = 1).set_index(
               ['actor_id','created_at_month_year','repo_id']).drop([
               'treatment','greater_1000_stars','greater_500_stars'], axis = 1),
           df_copilot_pr_lengths_grouped.fillna(0).set_index(['actor_id','created_at_month_year','repo_id']).rename(
               {'hours':'avg_hours_per_pr', 'hours_per_commit':'avg_hours_per_pr_commit', 
                'hours_per_commit_change':'avg_hours_per_pr_commit_change',
               'commit_count':'pr_commit_count', 'commit changes': 'pr_length_commit_changes',
               'code_hours':'pr_code_hours','code_hours_per_commit':'code_hours_per_pr_commit',
               'code_hours_per_commit_change':'code_hours_per_pr_commit_change', }, axis = 1).drop([
               'treatment','greater_1000_stars','greater_500_stars'], axis = 1),
           df_copilot_push_grouped.fillna(0).rename({'commit_actor_id':'actor_id'}, axis = 1).set_index(
               ['actor_id','created_at_month_year','repo_id']).rename(
               {'commit changes total':'push_commit_changes_total','commit additions':'push_commit_additions',
                'commit deletions':'push_commit_deletions', 'push_ref_changes':'push_main_commit_changes',
               'push_ref_additions':'push_main_commit_additions', 'push_ref_deletions':'push_main_commit_deletions'}, axis = 1).drop([
               'treatment','greater_1000_stars','greater_500_stars'], axis = 1),
           df_copilot_prs_grouped.fillna(0).rename({'pr_user_id':'actor_id'}, axis = 1).set_index(
               ['actor_id','created_at_month_year','repo_id']).rename({
               'pr_number_count':'pr_count', 'linked_pr_sum':'issue_pr_count', 'merged_pr_sum':'merged_pr_count'}, axis = 1).drop([
               'treatment','greater_1000_stars','greater_500_stars'], axis = 1),
          ], axis = 1)

In [ ]:
df_all_data.rename({'pr_merged_sum':'commit_count_pr_merged',
         'push_main_sum': 'commit_count_push_main'}, axis = 1, inplace = True)

In [ ]:
agg_dict = {}
for col in df_all_data.columns[3:]:
    agg_dict[col] = 'sum'
agg_dict['repo_id'] = 'count'

In [ ]:
df_all_data = df_all_data.reset_index()
df_all_data.fillna('NaN')
df_all_data['active_repos'] = df_all_data.groupby(['actor_id','created_at_month_year'])['repo_id'].transform('count')
df_all_data = df_all_data.groupby(['actor_id','created_at_month_year','active_repos','repo_id'], dropna = False).sum(min_count=1).reset_index()

In [ ]:
df_all_data_final = pd.merge(
    df_all_data,#[(~df_all_data['commit_count_pr'].isna())],
    population[['actor_id','repo_id','Free_Copilot']].drop_duplicates().rename(
             {'Free_Copilot':'treatment'}, axis = 1))

In [ ]:
df_fork_total = df_fork[['created_at_month_year','repo_id']].value_counts().reset_index().sort_values(
    'created_at_month_year').rename({'count':'fork_count'}, axis = 1)
df_fork_total['cumulative_forks'] = df_fork_total.groupby('repo_id').transform('cumcount')

In [ ]:
df_watch_total = df_watch[['created_at_month_year','repo_id']].value_counts().reset_index().sort_values(
    'created_at_month_year').rename({'count':'watch_count'}, axis = 1)
df_watch_total['cumulative_watches'] = df_watch_total.groupby('repo_id').transform('cumcount')

In [ ]:
df_all_data_final['month'] = df_all_data_final['created_at_month_year'].dt.month
df_all_data_final['appearances'] = df_all_data_final.sort_values('created_at_month_year').groupby(
    ['actor_id','repo_id']).transform('cumcount')
df_all_data_final['coworkers'] = df_all_data_final.groupby(['repo_id','created_at_month_year'])['month'].transform('count')

In [ ]:
# how many issues did they open
# how many issues did they comment on?
# how many issues is the project dealing with
issueEventData['created_at_month_year'] = issueEventData['created_at'].apply(lambda x: datetime.datetime(x.year, x.month, 28, 0 ,0, 0, tzinfo = pytz.UTC))
repo_issue_opened = issueEventData[issueEventData['issue_action'] == 'opened'].groupby(
    ['repo_id', 'created_at_month_year'])['created_at'].count().rename('repo_issues_opened').reset_index()
repo_issue_closed = issueEventData[issueEventData['issue_action'] == 'closed'].groupby(
    ['repo_id', 'created_at_month_year'])['created_at'].count().rename('repo_issues_closed').reset_index()
user_issue_handling = issueEventData[issueEventData['issue_action'].isin(['opened','closed'])].groupby(
    ['actor_id','repo_id', 'created_at_month_year'])['created_at'].count().rename('issues_managed').reset_index()
user_issue_comments = issueEventData[issueEventData['issue_action'] == None].groupby(
    ['actor_id','repo_id', 'created_at_month_year'])['created_at'].count().rename('comments_made').reset_index()



In [ ]:
df_all_data_final = pd.merge(pd.merge(df_all_data_final, df_fork_total, how = 'left'), df_watch_total, how = 'left')

In [ ]:
df_all_data_final = pd.merge(pd.merge(pd.merge(pd.merge(df_all_data_final, 
                                               repo_issue_opened, how = 'left'), user_issue_handling, how = 'left'),
                             user_issue_comments, how = 'left'),
                             repo_issue_closed, how = 'left')

In [ ]:
df_all_data_final[['repo_issues_opened','repo_issues_closed','issues_managed','comments_made']] = df_all_data_final[
    ['repo_issues_opened','repo_issues_closed','issues_managed','comments_made']].fillna(0)

In [ ]:
df_all_data_final['repos_2000'] = df_all_data_final['repo_id'].isin(repos2000)
df_all_data_final['repos_1000'] = df_all_data_final['repo_id'].isin(repos1000)
df_all_data_final['repos_500'] = df_all_data_final['repo_id'].isin(repos500)

In [ ]:
df_all_data_final = pd.read_parquet('results/data/df_final.parquet')

In [ ]:
df_all_dates = df_all_data_final.groupby(['actor_id','repo_id'])['created_at_month_year'].min().rename('earliest_appearance').reset_index()
df_all_dates['created_at_month_year'] = df_all_dates['earliest_appearance'].apply(
    lambda x: pd.date_range(x - pd.DateOffset(days=27), 
                            datetime.datetime(2024, 2, 28, tzinfo = pytz.UTC), freq='MS') + pd.DateOffset(days=27))
df_all_dates = df_all_dates.explode('created_at_month_year').drop('earliest_appearance', axis = 1)

In [ ]:
df_all_dates['df_all_dates'] = 1

In [ ]:
df_all_data_final['earliest_date'] = df_all_data_final.groupby(['actor_id','repo_id'])['created_at_month_year'].transform('min')
df_all_data_final['latest_date'] = df_all_data_final.groupby(['actor_id','repo_id'])['created_at_month_year'].transform('max')

In [ ]:
df_all_data = pd.concat([df_all_dates.set_index(['actor_id','repo_id','created_at_month_year']),
                         df_all_data_final.set_index(['actor_id','repo_id','created_at_month_year'])], axis = 1).reset_index()
df_all_data = df_all_data[df_all_data['created_at_month_year'].dt.year<2025]
df_all_data[['earliest_date','latest_date']] = df_all_data.sort_values(
    ['actor_id','repo_id','created_at_month_year'])[['earliest_date','latest_date']].fillna(method = 'ffill')
df_all_data = df_all_data[df_all_data['created_at_month_year']>=df_all_data['earliest_date']]
df_all_data = df_all_data[df_all_data['created_at_month_year']<=df_all_data['latest_date']]
df_all_data[['repos_2000','repos_1000','repos_500']] = df_all_data[['repos_2000','repos_1000','repos_500']].fillna(method = 'ffill')
df_all_data_2 = df_all_data.fillna(0)

In [ ]:
df_all_data_2.to_parquet('results/data/df_final.parquet')

In [ ]:
df_all_data_2.rename({'pr_merged_sum':'commit_count_pr_merged',
         'push_main_sum': 'commit_count_push_main'}, axis = 1, inplace = True)

In [ ]:
df_all_data_2.columns

In [ ]:
df_all_data_2[df_all_data_2['push_length_commit_changes'] != 0]